### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты)

In [53]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import re
import pandas as pd
import json
from pymongo import MongoClient
from selenium import webdriver
import time
from pprint import pprint

In [54]:
client = MongoClient('localhost', 27017)
db = client['pacring_job_sites_db']
collection = db['hh_superjobs']

In [55]:
# Определяем параметры

vac = input("Какую должность ищем: ")
max_list_hh = int(input("Сколько страниц HH анализировать: "))

Какую должность ищем: python
Сколько страниц HH анализировать: 1


In [56]:

cur_req = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")    
cur = json.loads(cur_req.text)  

In [57]:
# Забираем инфу с HH.ru



for x in range(max_list_hh):
    driver = webdriver.Chrome(executable_path=r"chromedriver.exe")
    driver.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text={vac}&from=suggest_post&page={x}')
    time.sleep(5)
    html = driver.page_source
    #resphh = requests.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text={vac}&from=suggest_post&page={x}', headers = {'User-agent':'Chrome/63.0.3239.108'}).text
    soup = bs(html, 'html.parser')
    cards = soup.find_all(name='div', attrs={"class" : "vacancy-serp-item"} )
    for i in cards:
        name_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).text
        
        link_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).get('href')
        
        employer = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-employer"}).text
        employer_link = f"https://hh.ru{i.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-employer'}).get('href')}"
        
        address = i.find('div', attrs = {'data-qa': "vacancy-serp__vacancy-address"}).text
        
        cost = i.find('span', attrs = {'data-qa' : "vacancy-serp__vacancy-compensation"})
        min_cost = np.nan
        max_cost = np.nan
        if cost:
            cost = cost.text 
            cost = cost.replace("\u202f", "").replace(" ", "")
            if re.findall('^от[0-9]+', cost):
                min_cost = int(re.findall('[0-9]+', cost)[0])
            elif re.findall('^до[0-9]+', cost):
                max_cost = int(re.findall('[0-9]+', cost)[0])
            else:
                min_cost = int(re.findall('[0-9]+', cost)[0])
                max_cost = int(re.findall('[0-9]+', cost)[1])
            if not re.findall('руб.', cost):
                currency = re.findall('[A-Z]{2,5}', cost)[0]
                if not np.isnan(min_cost):
                    min_cost = int(min_cost*cur["Valute"][currency]["Value"])//1000*1000
                if not np.isnan(max_cost):
                    max_cost = int(max_cost*cur["Valute"][currency]["Value"])//1000*1000
        #Собираем серию
        
        job_info = {}
        job_info['Наименование должности'] = name_job
        job_info['Ссылка на позицию'] = link_job
        job_info['Минимальная ЗП'] = min_cost
        job_info['Максимальная ЗП'] = max_cost
        job_info['Сайт сбора позиции'] = "hh.ru"
        job_info['Работадатель'] = employer
        job_info['Ссылка на работодателя'] = employer_link
        job_info['Место работы'] = address
        
        
        #Вставляем в БД
        
        if not collection.find_one({'Ссылка на позицию' : job_info['Ссылка на позицию']}):
            collection.insert_one(job_info)        

<ipython-input-57-af773dd646cd>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"chromedriver.exe")


In [58]:
my_salary = int(input("От какой ЗП выводить: "))

От какой ЗП выводить: 250000


In [59]:
for i in collection.find({ "$or" : [{'Минимальная ЗП':{"$gt": my_salary}}, {'Максимальная ЗП':{"$gt": my_salary}}]}):
    pprint(i)

{'_id': ObjectId('61b630c834d3b70f94cd4160'),
 'Максимальная ЗП': 600000,
 'Место работы': 'Москва',
 'Минимальная ЗП': 350000,
 'Наименование должности': 'Ведущий разработчик Python',
 'Работадатель': 'HR CITY MOSCOW',
 'Сайт сбора позиции': 'hh.ru',
 'Ссылка на позицию': 'https://hh.ru/vacancy/50062453?from=vacancy_search_list&query=python',
 'Ссылка на работодателя': 'https://hh.ru/employer/4187159'}
{'_id': ObjectId('61b630c834d3b70f94cd4164'),
 'Максимальная ЗП': 280000,
 'Место работы': 'Москва',
 'Минимальная ЗП': 200000,
 'Наименование должности': 'Python разработчик (Django)',
 'Работадатель': 'АО\xa0Семейный доктор',
 'Сайт сбора позиции': 'hh.ru',
 'Ссылка на позицию': 'https://hh.ru/vacancy/49789781?from=vacancy_search_list&query=python',
 'Ссылка на работодателя': 'https://hh.ru/employer/142683'}
{'_id': ObjectId('61b630c834d3b70f94cd4165'),
 'Максимальная ЗП': nan,
 'Место работы': 'Москва',
 'Минимальная ЗП': 600000,
 'Наименование должности': 'Разработчик Solidity/Разра